In [1]:
import numpy as np
import pandas as pd
from numpy import array
from numpy import argmax
import sys
#!{sys.executable} -m  pip install --upgrade tensorflow
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, GRU, Bidirectional, MaxPooling1D, Conv1D, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform, he_uniform, zeros
from keras.optimizers import SGD, Adam, Adadelta
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras import metrics

C:\Users\mehak\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def import_textFile(filePath):
    ds_text = []
    with open(filePath,'rb') as f:
        for idx,ln in enumerate(f):
            decoded=False
            line=''
            for cp in ('cp1252', 'cp850','utf-8','utf8'):
                try:
                    line = ln.decode(cp)
                    decoded=True
                    break
                except UnicodeDecodeError:
                    pass
            if decoded:
                ds_text.insert(idx, line.rstrip())
    return ds_text

In [35]:
train_ds = pd.read_csv('Dataset/x_train.csv', sep=",", header=None,index_col = False)
test_ds = pd.read_csv('Dataset/x_test.csv', sep=",", header=None,index_col = False)

x_train = train_ds[0]
y_train = train_ds[1]

x_test = test_ds[0]
y_test = test_ds[1]

In [38]:
maxLen = len(max(x_train, key=len).split())
print(maxLen)

17


In [39]:
def get_one_hot(y):
    one_hot_temp_array = array(y)
    one_hot_array = to_categorical(one_hot_temp_array)
    return one_hot_array
# invert encoding
#inverted = argmax(encoded[0])

In [9]:
#load glove diembedding_dict = dict()
embedding_dict = dict()
glove_words = []
#with open('Dataset/glove.twitter.27B/glove.twitter.27B.25d.txt','rb') as f:
with open('Dataset/glove.6B.50d.txt','rb') as f:
    for ln in f:
        decoded=False
        line=''
        for cp in ('cp1252', 'cp850','utf-8','utf8'):
            try:
                line = ln.decode(cp)
                decoded=True
                break
            except UnicodeDecodeError:
                pass
        if decoded:
            split = line.split(' ')

            word = split[0]
            glove_words.append(word)

            features = split[1:]
            features = np.array(
                [float(val) for val in features]
            )
            embedding_dict[word] = features


In [10]:
number_to_word = glove_words
word_to_number = dict((word,idx) for idx,word in enumerate(glove_words))

In [11]:
def get_indices(input_x, word_to_number, max_len):
    m = input_x.shape[0]
    x_indices = np.zeros((m,max_len))

    for i in range(m):
        word_list =input_x.iloc[i][0].lower().split()
        for idx,word in enumerate(word_list):
            if (idx ==32):
                continue
            x_indices[i, idx] = word_to_number[word]
    return x_indices

In [12]:
def get_embedding_layer(embedding_dict, word_to_number):
    
    emb_shape = embedding_dict["cucumber"].shape[0]
    total_words = len(word_to_number) + 1
    
    emb_matrix = np.zeros((total_words,emb_shape))
    
    for word,idx in word_to_number.items():
        emb_matrix[idx, :] = embedding_dict[word]

    embedding_layer = Embedding(total_words, emb_shape, trainable = True)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [76]:
def senti_model(input_shape,embedding_dict, word_to_number):
    np.random.seed(1)
    sentence_indices = Input(shape = input_shape, dtype = np.int32)
    embedding_layer = get_embedding_layer(embedding_dict, word_to_number)
    
    embeddings = embedding_layer(sentence_indices)   
    print(embeddings)
    #,dropout=0.3,recurrent_dropout=0.3)
    X = Dropout(0.4)(embeddings)
    X = Bidirectional(GRU(64,return_sequences=True))(X)
    #X = Dropout(0.4)(X)
    X = Bidirectional(GRU(64,return_sequences=True))(X)
    #X = Bidirectional(GRU(64,return_sequences=True))(X)
    #X = Dropout(0.5)(X)
    #X = Bidirectional(GRU(64,return_sequences=True))(X)
    #X = Dropout(0.5)(X)
    #X = BatchNormalization()
    #X = BatchNormalization()(X)
    X= Conv1D(50, 3, activation='relu',kernel_regularizer=regularizers.l2(0.02))(X)
    X = MaxPooling1D(2)(X)
    X = Dense(16, activation='elu')(X)
    X = Dropout(0.4)(X)
    #l_cov2 = Conv1D(32, 3, activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_drop1)
    #l_pool2 = MaxPooling1D(2)(l_cov2)
    #l_drop2 = Dropout(0.3)(l_pool2)
    #X = Flatten()(X)
    X = Bidirectional(GRU(64,return_sequences=False))(X)
    X = BatchNormalization()(X)
    X = Dense(64, activation='elu')(X)
    #X = Dense(64, activation='elu', name='fc-1')(X)
    X = Dense(2, activation='softmax', name='fc')(X)


    #X = Activation('softmax')(X)
    
    model = Model(inputs = sentence_indices, outputs = X, name='stage-1')
    
    return model

In [77]:
model = senti_model((maxLen,), embedding_dict, word_to_number)
model.summary()

Tensor("embedding_12/embedding_lookup/Identity:0", shape=(?, 17, 50), dtype=float32)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 17)                0         
_________________________________________________________________
embedding_12 (Embedding)     (None, 17, 50)            20000050  
_________________________________________________________________
dropout_3 (Dropout)          (None, 17, 50)            0         
_________________________________________________________________
bidirectional_29 (Bidirectio (None, 17, 128)           44160     
_________________________________________________________________
bidirectional_30 (Bidirectio (None, 17, 128)           74112     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 15, 50)            19250     
_________________________________________________________

In [78]:
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
adam = Adam(lr=0.0001)
adadelta = Adadelta(lr=1.00, rho=0.95, epsilon=None, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=adadelta, metrics=[metrics.binary_accuracy])

In [79]:
x_train_indices = get_indices(x_train, word_to_number, maxLen)
y_train_oh = get_one_hot(y_train)
print(y_train_oh.shape)

(217, 2)


In [80]:
model.fit(x_train_indices, y_train_oh, epochs = 100, batch_size = 16, shuffle=True)

Epoch 1/100
217/217 [==============================] - 21s 97ms/step - loss: 2.3033 - binary_accuracy: 0.4977
Epoch 2/100
217/217 [==============================] - 8s 35ms/step - loss: 2.0173 - binary_accuracy: 0.4885
Epoch 3/100
217/217 [==============================] - 7s 33ms/step - loss: 1.8931 - binary_accuracy: 0.4654
Epoch 4/100
217/217 [==============================] - 7s 33ms/step - loss: 1.7447 - binary_accuracy: 0.5253
Epoch 5/100
217/217 [==============================] - 7s 34ms/step - loss: 1.5059 - binary_accuracy: 0.6175
Epoch 6/100
217/217 [==============================] - 7s 33ms/step - loss: 1.5619 - binary_accuracy: 0.4700
Epoch 7/100
217/217 [==============================] - 7s 33ms/step - loss: 1.4212 - binary_accuracy: 0.4977
Epoch 8/100
217/217 [==============================] - 7s 34ms/step - loss: 1.3261 - binary_accuracy: 0.5484
Epoch 9/100
217/217 [==============================] - 7s 32ms/step - loss: 1.2682 - binary_accuracy: 0.5023
Epoch 10/100
217/2

KeyboardInterrupt: 

In [30]:
x_test_indices = get_indices(x_test, word_to_number, max_len = maxLen)
y_test_oh = get_one_hot(y_test)
loss, acc = model.evaluate(x_test_indices, y_test_oh)
print()
print("Test accuracy = ", acc)

143/143 [==============================] - 3s 20ms/step

Test accuracy =  0.741258742926004


In [33]:
def get_emotion(value):
    unique_labels = ["negative","positive"]
    return unique_labels[value]

In [34]:
C = 4
y_test_array = y_test.values
y_test_oh = np.eye(C)[y_test_array.reshape(-1)]
X_test_indices = get_indices(x_test, word_to_number, maxLen)
pred = model.predict(x_test_indices)
for i in range(len(x_test)):
    x = x_test_indices
    num = np.argmax(pred[i])
    if(num != y_test_array[i]):
        print('Expected '+ get_emotion(y_test_array[i]) + ' prediction: '+ get_emotion(num) + ' ' + x_test[i] )

Expected positive prediction: negative thank much gloria series sweet thoughtful you made day joyful i love too of
Expected positive prediction: negative thanks making mass fr vallely respectful meaningful joyful
Expected positive prediction: negative currently listening amp podcasts can guys please move yvr hilarious mission
Expected positive prediction: negative well done ladies a great award amazing team a delight present award you
Expected positive prediction: negative use smile change world series let world change smile quote love fun
Expected positive prediction: negative a a a whole time watching lost glasses it hilarious
Expected positive prediction: negative i simple human really loves like truly madly deeply rejoicing existence class queen dom
Expected positive prediction: negative ill there cant wait
Expected positive prediction: negative series meant be happy happy
Expected positive prediction: negative thought gets day thought smile never fails making smile a
Expected posi